In [ ]:
import os
os.chdir('..')

In [ ]:
from datetime import timezone
from pathlib import Path

import polars as pl
from dateutil import parser
from datetime import timedelta

from config import DataFrequency, BINANCE_DATA_DIR, TradeType

from util.ts_manager import TSManager
from aws.kline.parse import read_csv
from aws.client_async import AwsKlineClient
from merge.kline import merge_klines

pl.Config.set_tbl_rows(30)

In [ ]:
symbol = 'BNXUSDT'
time_interval = '1m'

In [ ]:
def scan_gaps(df, min_delta, min_price_chg):
    df = df.lazy()

    df = df.with_columns(pl.col('candle_begin_time').diff().alias('time_diff'))
    df = df.with_columns((pl.col('open') / pl.col('close').shift() - 1).alias('price_change'))


    df_gap = df.filter((pl.col('time_diff') > min_delta) & (pl.col('price_change').abs() > min_price_chg))
    df_gap = df_gap.select('candle_begin_time', 'open', 'time_diff', 'price_change')

    return df_gap


min_delta = timedelta(days=1)
min_price_chg = 0.1

df = merge_klines(TradeType.spot, 'BNXUSDT', time_interval, exclude_empty=True)
display(scan_gaps(df, min_delta, min_price_chg))

df = merge_klines(TradeType.spot, 'BTCUSDT', time_interval, exclude_empty=True)
display(scan_gaps(df, min_delta, min_price_chg))


In [ ]:
min_delta = timedelta(days=1)
min_price_chg = 0.1
trade_type = TradeType.spot

symbols = (BINANCE_DATA_DIR / "parsed_data" / trade_type.value / "klines").glob("*")
symbols = sorted(s.name for s in symbols)

dfs = []
for symbol in symbols:
    df = merge_klines(TradeType.spot, symbol, time_interval, exclude_empty=True)
    df_gap = scan_gaps(df, min_delta, min_price_chg)
    if not df_gap.is_empty():
        print(symbol)
        dfs.append(df_gap.with_columns(pl.lit(symbol).alias("symbol")))

pl.concat(dfs).sort('candle_begin_time')

In [ ]:
min_delta = timedelta(days=3)
min_price_chg = 0
trade_type = TradeType.spot

symbols = (BINANCE_DATA_DIR / "parsed_data" / trade_type.value / "klines").glob("*")
symbols = sorted(s.name for s in symbols)

dfs = []
for symbol in symbols:
    df = merge_klines(TradeType.spot, symbol, time_interval, exclude_empty=True)
    df_gap = scan_gaps(df, min_delta, min_price_chg)
    if not df_gap.is_empty():
        print(symbol)
        dfs.append(df_gap.with_columns(pl.lit(symbol).alias("symbol")))

pl.concat(dfs).sort('candle_begin_time')

In [ ]:
min_delta = timedelta(days=1)
min_price_chg = 0.1
trade_type = TradeType.um_futures

symbols = (BINANCE_DATA_DIR / "parsed_data" / trade_type.value / "klines").glob("*")
symbols = sorted(s.name for s in symbols)

dfs = []
for symbol in symbols:
    df_gap = scan_gaps(trade_type, symbol, time_interval, min_delta, min_price_chg)
    if not df_gap.is_empty():
        print(symbol)
        dfs.append(df_gap.with_columns(pl.lit(symbol).alias("symbol")))

pl.concat(dfs)

In [ ]:
min_delta = timedelta(days=3)
min_price_chg = 0
trade_type = TradeType.um_futures

symbols = (BINANCE_DATA_DIR / "parsed_data" / trade_type.value / "klines").glob("*")
symbols = sorted(s.name for s in symbols)

dfs = []
for symbol in symbols:
    df_gap = scan_gaps(trade_type, symbol, time_interval, min_delta, min_price_chg)
    if not df_gap.is_empty():
        print(symbol)
        dfs.append(df_gap.with_columns(pl.lit(symbol).alias("symbol")))

pl.concat(dfs)